In [ ]:
from pathlib import Path
import _fixpath

DAY_DIR = Path("/media/ko/MAUI63/2025-11-21-rotorua-test-flight/2025-11-21")
GPS_DIR = DAY_DIR / "gps"
CAMERA_DIR = DAY_DIR / "cameras"
GROUND_LEVEL_ALTITUDE_METERS = 940 / 3.333  # Rotorua at 940ft


In [27]:
from collections import namedtuple
from datetime import datetime, timedelta
import pytz

GpsTriggerEvent = namedtuple("GpsTriggerEvent", ["time", "lat", "lon", "agl", "num_satellites", "line_idx"])

# Parse the GPS data:
gps_points = []
for fpath in sorted(GPS_DIR.glob("*.CAM")):
    # Parse the start time from the filename:
    # Read the files:
    with open(fpath, "r") as f:
        for line_idx, line in enumerate(f):
            parts = line.split(",")
            week = int(parts[0])
            start_time = datetime(1980, 1, 6, tzinfo=pytz.utc) + timedelta(weeks=week)
            milliseconds_since_start_of_week = float(parts[1])
            num_satellites = int(parts[7])
            lat = float(parts[8])
            lon = float(parts[9])
            agl = float(parts[10]) - GROUND_LEVEL_ALTITUDE_METERS
            time = start_time + timedelta(milliseconds=milliseconds_since_start_of_week)
            gps_points.append(GpsTriggerEvent(time, lat, lon, agl, num_satellites, line_idx))
print(f"Parsed {len(gps_points)} GPS trigger events.")


Parsed 1562 GPS trigger events.


In [28]:
# Plot the GPS points interactively with hover info:
import plotly.graph_objects as go


def make_plot(gps_points):
    lats = [p.lat for p in gps_points]
    lons = [p.lon for p in gps_points]
    alts = [p.agl for p in gps_points]

    fig = go.Figure(
        data=[
            go.Scatter3d(
                x=lons,
                y=lats,
                z=alts,
                mode="lines+markers",
                marker=dict(size=4),
                text=[f"Index: {i}\nAlt: {alts[i]}" for i in range(len(gps_points))],
                hoverinfo="text",
            )
        ]
    )
    fig.update_layout(
        scene=dict(xaxis_title="Longitude", yaxis_title="Latitude", zaxis_title="Altitude"),
        title="GPS Track",
        width=800,
        height=800,
        margin=dict(l=0, r=0, b=0, t=40),
    )
    fig.show()


make_plot(gps_points)

In [ ]:
assert False, "Don't manually run this!"

# Manually remove any early triggers etc.

indices_to_ignore = []
gps_points = [p for i, p in enumerate(gps_points) if i not in indices_to_ignore]
print(f"Total GPS points: {len(gps_points)}")

make_plot(gps_points)

In [7]:
from img_utils import get_a7rv_image_timestamp, get_exif_lazy
from tqdm import tqdm

# Get all the images per camera:
ImageMeta = namedtuple("ImageMeta", ["path", "timestamp", "aperture", "shutter_speed", "iso"])

NZDT = pytz.timezone("Pacific/Auckland")

cameras = {}
for camera_dir in CAMERA_DIR.iterdir():
    camera = camera_dir.name
    if camera not in ("r09", "r28", "l09", "l28"):
        continue
    assert camera in ("r09", "r28", "l09", "l28")
    images = []
    for img_path in tqdm(list(sorted(camera_dir.rglob("*.JPG"))), desc=f"Camera {camera}"):
        exifimg = get_exif_lazy(img_path, chunksize=64000)
        timestamp = get_a7rv_image_timestamp(exifimg)
        aperture = exifimg["f_number"]
        shutter_speed = exifimg["exposure_time"]
        iso = exifimg["photographic_sensitivity"]
        if timestamp.tzinfo is None:
            # Assume NZDT:
            timestamp = NZDT.localize(timestamp)
        images.append(
            ImageMeta(path=img_path, timestamp=timestamp, aperture=aperture, shutter_speed=shutter_speed, iso=iso)
        )

    # Important to sort by timestamp - we assume this later!
    cameras[camera] = sorted(images, key=lambda c: c.timestamp)


Camera r28: 100%|██████████| 787/787 [00:38<00:00, 20.42it/s]


In [29]:
# Plot unaligned:
import plotly.graph_objects as go


def plot_aligned_data(gps_points, cameras, camera_offsets=None):
    fig = go.Figure()

    timestamps = [p.time.astimezone(NZDT) for p in gps_points]
    fig.add_trace(
        go.Scatter(
            x=timestamps,
            y=[1] * len(timestamps),
            mode="markers",
            name="GPS Points",
            marker=dict(size=4, color="blue"),
        )
    )

    # Plot camera points:
    colors = {"r09": "red", "r28": "green", "l09": "orange", "l28": "purple"}
    for idx, (cam, images) in enumerate(cameras.items()):
        offset = timedelta(seconds=camera_offsets[cam]) if camera_offsets is not None else timedelta(seconds=0)
        timestamps = [img.timestamp + offset for img in images]
        fig.add_trace(
            go.Scatter(
                x=timestamps,
                y=[1 + (idx + 1) / 10] * len(timestamps),
                mode="markers",
                name=cam,
                marker=dict(size=2, color=colors[cam]),
            )
        )

    fig.show()


plot_aligned_data(gps_points, cameras, None)

In [30]:
# OK, looks like we can just align using the first point after 2025-11-21 13:10 NZDT:
camera_offsets = {}
cutoff = NZDT.localize(datetime(2025, 11, 21, 13, 10))
for cam, images in cameras.items():
    gps_timestamps = sorted([gps.time.timestamp() for gps in gps_points if gps.time >= cutoff])
    image_timestamps = sorted([img.timestamp.timestamp() for img in images if img.timestamp >= cutoff])
    offset = gps_timestamps[0] - image_timestamps[0]
    print(f"Camera {cam} offset: {offset} seconds")
    camera_offsets[cam] = offset

plot_aligned_data(gps_points, cameras, camera_offsets)

Camera l09 offset: 50.89800000190735 seconds
Camera l28 offset: 48.24600005149841 seconds
Camera r09 offset: 51.28600001335144 seconds
Camera r28 offset: 23.4060001373291 seconds


In [31]:
max_time_diff_s = 0.25  # This is fine tuned somewhat

# Precompute sorted timestamps for each camera
sorted_camera_timestamps = {
    cam: [img.timestamp + timedelta(seconds=camera_offsets[cam]) for img in images] for cam, images in cameras.items()
}
matched_images = {cam: set() for cam in cameras.keys()}
output = []
camera_sorted_idxs = {cam: 0 for cam in cameras.keys()}

for gps_point in tqdm(gps_points):
    gps_time = gps_point.time
    matched = {}
    for cam, idx in camera_sorted_idxs.items():
        timestamps = sorted_camera_timestamps[cam]
        while True:
            if idx >= len(timestamps):
                break
            t = timestamps[idx]
            if (t - gps_time).total_seconds() < -max_time_diff_s:
                # Too early, discard this timestamp
                idx += 1
            elif (t - gps_time).total_seconds() > max_time_diff_s:
                # Gone too far
                break
            else:
                assert abs((t - gps_time).total_seconds()) <= max_time_diff_s
                matched[cam] = (t, cameras[cam][idx])
                idx += 1
                break
        camera_sorted_idxs[cam] = idx
    # print(camera_sorted_idxs)

    output.append((gps_point, matched))
output[0]

100%|██████████| 1562/1562 [00:00<00:00, 104283.44it/s]


(GpsTriggerEvent(time=datetime.datetime(2025, 11, 20, 23, 57, 4, 783000, tzinfo=<UTC>), lat=-38.108082056, lon=176.318081566, agl=25.484997179717993, num_satellites=16, line_idx=0),
 {'l09': (datetime.datetime(2025, 11, 21, 12, 57, 4, 638000, tzinfo=<DstTzInfo 'Pacific/Auckland' NZDT+13:00:00 DST>),
   ImageMeta(path=PosixPath('/media/ko/MAUI63/2025-11-21-rotorua-test-flight/2025-11-21/cameras/l09/10251121/_09L9114.JPG'), timestamp=datetime.datetime(2025, 11, 21, 12, 56, 13, 740000, tzinfo=<DstTzInfo 'Pacific/Auckland' NZDT+13:00:00 DST>), aperture=4.0, shutter_speed=0.0004, iso=2500)),
  'l28': (datetime.datetime(2025, 11, 21, 12, 57, 4, 678000, tzinfo=<DstTzInfo 'Pacific/Auckland' NZDT+13:00:00 DST>),
   ImageMeta(path=PosixPath('/media/ko/MAUI63/2025-11-21-rotorua-test-flight/2025-11-21/cameras/l28/10251121/_28L9119.JPG'), timestamp=datetime.datetime(2025, 11, 21, 12, 56, 16, 432000, tzinfo=<DstTzInfo 'Pacific/Auckland' NZDT+13:00:00 DST>), aperture=2.8, shutter_speed=0.0004, iso=16

In [32]:
def plot_aligned_data(matched, gps_points, cameras, camera_offsets):
    fig = go.Figure()

    offsets = {"l09": 0.1, "l28": 0.2, "r09": 0.3, "r28": 0.4}

    # Plot unmatched gps:
    matched_gps_times = [p.time.astimezone(NZDT) for p, cam_dict in matched if not cam_dict]
    fig.add_trace(
        go.Scatter(
            x=matched_gps_times,
            y=[1] * len(matched_gps_times),
            mode="markers",
            name="GPS Point",
            marker=dict(size=8, color="red"),
            opacity=0.5,
        )
    )

    # And unmatched camera points:
    for cam in cameras.keys():
        all_ts = set(sorted_camera_timestamps[cam])
        matched_ts = []
        for gps_point, cam_dict in matched:
            if len(cam_dict) == 0 or cam not in cam_dict:
                continue
            matched_ts.append(cam_dict[cam][0])
        unmatched_ts = sorted(all_ts - set(matched_ts))
        ts = [t.astimezone(NZDT) for t in unmatched_ts]
        fig.add_trace(
            go.Scatter(
                x=ts,
                y=[1 + offsets[cam]] * len(ts),
                mode="markers",
                name=cam,
                marker=dict(size=8, color="red"),
                opacity=0.5,
            )
        )

    timestamps = [p.time.astimezone(NZDT) for p in gps_points]
    fig.add_trace(
        go.Scatter(
            x=timestamps,
            y=[1] * len(timestamps),
            mode="markers",
            name="GPS Points",
            marker=dict(size=4, color="blue"),
        )
    )

    # Plot camera points:
    colors = {"r09": "red", "r28": "green", "l09": "orange", "l28": "purple"}
    for idx, (cam, images) in enumerate(cameras.items()):
        offset = timedelta(seconds=camera_offsets[cam])
        timestamps = [img.timestamp + offset for img in images]
        fig.add_trace(
            go.Scatter(
                x=timestamps,
                y=[1 + offsets[cam]] * len(timestamps),
                mode="markers",
                name=cam,
                marker=dict(size=2, color=colors[cam]),
            )
        )

    fig.show()


plot_aligned_data(output, gps_points, cameras, camera_offsets)

In [33]:
# What are the counts of frames per GPS point?
from collections import Counter

counts = Counter()
for gps_point, cam_dict in output:
    counts[len(cam_dict)] += 1
print(counts)

Counter({4: 784, 0: 778})


In [34]:
import json
import constants

# Phew, we're done! Let's create an output json:
json_output = {
    "cameras": list(cameras.keys()),
    "frames": [],
}
for gps_point, cam_dict in output:
    entry = {
        "lat": gps_point.lat,
        "lon": gps_point.lon,
        "agl": gps_point.agl,
        "gps_time": gps_point.time.isoformat(),
        "images": {
            cam: dict(path=str(img_meta.path.relative_to(DAY_DIR)), t=t.isoformat())
            for cam, (t, img_meta) in cam_dict.items()
        },
    }
    json_output["frames"].append(entry)
opath = DAY_DIR / "frames.json"
with open(opath, "w") as f:
    json.dump(json_output, f, indent=2)